# Einführung in das Clustering

## Grundsätzlicher Ansatz des Clusterings

<img style="float: right; padding-left: 10%; padding-right: 5%;" src="images/example.png" width="40%">

### Aufgabenstellung
* Gegeben eine Menge an Datenpunkten
* Erkenne interessante Strukturen in den Daten
* Cluster = Menge von Datenpunkten die untereinander ähnlich sind, sich aber von den anderen Datenpunkten unterscheiden

### Zentrale Fragestellung
**Was bedeutet es, dass Datenpunkte "ähnlich" sind?**  
&rarr; Keine universelle Antwort  
&rarr; Kein universell gültiges Clustering

### Beispiele
* Kundenprofile
* Fehlerkonstellationen

# K-Means

<img style="float: right; padding-left: 50px; padding-right: 50px;" src="images/kmeans.png" width="50%">

Wir schauen uns nun **K-Means** an, das am häufigstens eingesetzte Clusteringverfahren.

* K-Means ist universell verwendbar
* Es basiert auf der Berechnung sogenannter Zentroide
* Die Zentroide entsprechen den Clustern
* Datenpunkte werden den jeweils nächsten Zentroiden zugeordnet
* Parameter: Anzahl der Cluster
* Das Verfahren läuft sehr schnell
* Die resultierenden Cluster sind konvex
* Eigentlich ein Algorithmus zur Partitionierung
* Clustering ist vollständig: jeder Datenpunkt wird einem Cluster zugeordnet  
&rarr; Das gilt allerdings auch für fehlerhafte Datenpunkte und Ausreißer

Nun schauen wir uns K-Means in Aktion an.

## Laden der Bibliotheken

In [ ]:
import os

import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.preprocessing import MinMaxScaler, StandardScaler, FunctionTransformer
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabasz_score
from sklearn.datasets import make_blobs

from ipywidgets import interact
import ipywidgets as widgets

from scipy.spatial import Voronoi, voronoi_plot_2d

import seaborn as snss

style = {'description_width': '150px'}
layout = widgets.Layout(width='400px')

## Vier synthetische Cluster

### Generierung und Visualisierung der Daten

Um die Funktionsweise des Clusterings zu prüfen, beginnen wir zuerst mit künstlich generierten Daten. Diese werden in vier Clustern verteilt, die um die Punkte (-1, -1), (-1, 1), (1, -1), (1, 1) zentriert sind und sich überlappen

In [ ]:
n = 100

data = np.concatenate(
    [np.random.multivariate_normal([x, y], np.diag([.3, .3]), size=n)
     for x in [-1, 1]
     for y in [-1, 1]]
)

In [ ]:
df = pd.DataFrame(data, columns=['x', 'y'])
_ = df.plot(kind='scatter', x='x', y='y', s=40, figsize=(6, 6))

## Arbeitsprinzip des Algorithmus

k-Means arbeitet iterativ:
1. Es werden _k_ Cluster-Repräsentanten (Zentroiden oder Centroids) initial ausgewählt. Diese Auswahl erfolgt je nach Implementierung rein zufällig oder (häufiger) anhand einer Auswahlheuristik zwecks schnellerer Konvergenz des Verfahrens.
2. Alle übrigen Datenpunkte werden dem Repräsentanten zugeordnet, dem sie - entsprechend dem definierten Ähnlichkeitsmaß - am nächsten sind.
3. Ein neues Set von _k_ Cluster-Repräsentanten wird erzeugt, indem für jedes Cluster der Durchschnittsvektor (Mean) der Attributwerte aller dem Cluster im vorhergehenden Schritt zugeordneten Datenpunkte berechnet wird.
4. Schritte 2 und 3 werden solange iterativ wiederholt, bis entweder eine vordefinierte Anzal $n_{max}$ von Iterationen durchlaufen wurde, oder in der $n$-ten Durchführung von Schritt 2 kein Datenpunkt mehr einem anderen Cluster zugeordnet wurde.

Nun definieren wir - ähnlich wie oben - einige Hilfsfunktionen, mit der wir interaktiv bestimmen und plotten können, wie das Zwischenergebnis nach einer vorgegebenen Anzahl von Iterationsschritten aussieht. Einige Schlüsselstellen im Code sind per Kommentar hervorgehoben.

In [ ]:
# Scatterplot mit Einfärbung der Punkte entsprechend ihrer Clusterzugehörigkeit,
# Zentroiden als schwarze Kreuze überlagert 
def plot_clustering_steps(df, clusterer, print_number=False):
    df = df.copy()
    df['cid'] = clusterer.fit_predict(df[['x', 'y']]) # Cluster(zwischen)ergebnis anhand übergeber Einstellung berechnen
    n_clusters = df['cid'].max() + 1
    n_outliers = np.sum(df['cid'] == -1)
    if print_number:
        print(f'number of clusters: {n_clusters}\nnumber of outliers: {n_outliers}')
    cmap = plt.get_cmap('Set1', n_clusters+1) # Einheitliche, vordefinierte Farbskala
    fig, ax = plt.subplots()
    
    # Plotten der Entscheidungsgrenzen der Zuordnung einzelner Punkte zu Zentroiden
    voro = Voronoi(clusterer.cluster_centers_)
    voronoi_plot_2d(voro, ax, show_points=False, show_vertices=False)
    
    # Plotten der Datenpunkte mit Einfärbung
    df.plot(ax=ax, kind='scatter', x='x', y='y', c='cid', cmap=cmap, s=40, colorbar=False, figsize=(6, 6))
    
    # Überlagern der Zentroiden als schwarze X-Symbole
    for c in clusterer.cluster_centers_:
        ax.plot(c[0], c[1], 'kX')
        
    ax.set_xlim(-3.0, 3.0)
    ax.set_ylim(-3.0, 3.0)
    
# k-Means mit fest vorgegebenen Inital-Zentroiden und bis zu einer vorgegebenen Anzahl Durchläufe berechnen
def stepped_kmeans(k, steps):
    kmeans = KMeans(n_clusters=k,
                    init=df[:k].to_numpy(), # Wähle die ersten k Zeilen des Dataframe als initiale Zentroiden
                    n_init=1, # Da die Zentroiden feststehen wird keine Auswahlheuristik benötigt
                    max_iter=steps   # Erzwinge Abbruch des Verfahrens nach der festglegten Anzahl Durchläufe
                   )
    plot_clustering_steps(df, clusterer=kmeans) # (Zwischen-)Ergebnis plotten

    
# Interaktionselemente mit der Berechnungs- und Plotfunktion verbinden
_ = interact(
    stepped_kmeans,
    k=widgets.SelectionSlider(
        value=4,
        options=range(3,15),
        layout=layout,
        style=style,
        description='Number of clusters (k)',
        orientation='horizontal'
    ),
    steps=widgets.SelectionSlider(
        options=range(1,15),
        layout=layout,
        style=style,
        description='Number of iterations',
        orientation='horizontal'
    )
)


Wir berechnen nun die zugehörige Silhouetten-Kurve. Wie man sehen kann, wird der höchste Wert bei 4, also der korrekten Anzahl der Cluster, erreicht.

In [ ]:
sil_values = []
for k in range(2,20):
    kmeans = KMeans(n_clusters=k)
    clust = kmeans.fit_predict(df[['x', 'y']])
    sil_values.append(silhouette_score(df[['x', 'y']], clust))

fig, ax = plt.subplots()
ax.plot(range(2,20),sil_values)
ax.set_xlabel('k')
ax.set_xticks(range(2, 20, 2))
ax.set_ylabel('Silhouette value');

## Detaillierter Silhouette Plot
Code snippet aus der scikit-learn Bibliothek

In [ ]:
# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X, y = make_blobs(
    n_samples=500,
    n_features=2,
    centers=4,
    cluster_std=1,
    center_box=(-10.0, 10.0),
    shuffle=True,
    random_state=42,
)  # For reproducibility

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example we
    # limit display to within [-0.2, 1]
    ax1.set_xlim([-0.2, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

## Grenzen von bzw. Annahmen hinter k-Means

Diese Beispiele aus den mit skLearn ausgelieferten Testdatensätze bzw. Dokumentation sollen Situationen veranschaulichen, in denen k-Means unintuitive und möglicherweise unerwartete Cluster erzeugt. In den ersten drei Diagrammen entsprechen die Eingabedaten nicht den impliziten Annahmen, welche k-Means zugrunde liegen, und als Ergebnis unerwünschte Cluster erzeugt werden. Im letzten Plot gibt k-means trotz ungleichmäßig großer Blobs intuitive Cluster zurück.

In [ ]:
plt.figure(figsize=(12, 12))

n_samples = 1500
random_state = 170
X, y = make_blobs(n_samples=n_samples, random_state=random_state)

# Falsch gewähltes K = "falsche" Anzahl Blobs 
y_pred = KMeans(n_clusters=2, random_state=random_state).fit_predict(X)

plt.subplot(221)
plt.scatter(X[:, 0], X[:, 1], c=y_pred)
plt.title("Falsches k ungleich der Anzahl Blobs")

# Anisotropicly distributed data
transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]
X_aniso = np.dot(X, transformation)
y_pred = KMeans(n_clusters=3, random_state=random_state).fit_predict(X_aniso)

plt.subplot(222)
plt.scatter(X_aniso[:, 0], X_aniso[:, 1], c=y_pred)
plt.title("Anisotropish verteilte, nicht kugelförmige Blobs")

# Different variance
X_varied, y_varied = make_blobs(n_samples=n_samples,
                                cluster_std=[1.0, 2.5, 0.5],
                                random_state=random_state)
y_pred = KMeans(n_clusters=3, random_state=random_state).fit_predict(X_varied)

plt.subplot(223)
plt.scatter(X_varied[:, 0], X_varied[:, 1], c=y_pred)
plt.title("Stark unterschiedliche Varianz (Punktdichte)")

# Unevenly sized blobs
X_filtered = np.vstack((X[y == 0][:500], X[y == 1][:100], X[y == 2][:10]))
y_pred = KMeans(n_clusters=3,
                random_state=random_state).fit_predict(X_filtered)

plt.subplot(224)
plt.scatter(X_filtered[:, 0], X_filtered[:, 1], c=y_pred)
plt.title("Blobs ungleicher Größe")

plt.show()

## Iris-Datenset

Als nächstes überprüfen wir die Funktionsweise von k-Means an einem echten Datensatz, dem berühmten Iris-Datensatz von Ronald Fisher (https://en.wikipedia.org/wiki/Iris_flower_data_set). Hierbei handelt es sich um Messungen von Blättern für unterschiedliche Arten aus der Gattung Iris (Schwertlilien).

Dieses wird zuerst aus der Datei "iris.csv" eingelesen und dann visualisiert. Dabei sind auf der X-Achse die Breite der Kelchblätter und auf der Y-Achse die Länge der Blütenblätter aufgetragen.

In [ ]:
dataset_path = os.environ['DATASET_PATH']
iris_df = pd.read_csv(dataset_path + '/Iris/iris.csv'). \
        rename({'sepal length (cm)': 's', 'sepal width (cm)': 'x', 'petal length (cm)': 'y', 'petal width (cm)': 'p'}, axis=1)
_ = iris_df.plot(kind='scatter', x='x', y='y', s=40, figsize=(6, 6))

### Clustering mit k-Means

In [ ]:
def fit_and_plot_clustering(df, clusterer, print_number=False):
    df = df.copy()
    df['clusterid'] = clusterer.fit_predict(df)
    n_clusters = df['clusterid'].max() + 1
    n_outliers = np.sum(df['clusterid'] == -1)
    if print_number:
        print(f'number of clusters: {n_clusters}\nnumber of outliers: {n_outliers}')
    cmap = plt.get_cmap('Set1', n_clusters+1)
    ax = df.plot(kind='scatter', x='x', y='y', c='clusterid', cmap=cmap, s=40, colorbar=False, figsize=(6, 6))
    ax.grid()

def plot_kmeans(k = 3):
    kmeans = KMeans(n_clusters=k, random_state=42)
    fit_and_plot_clustering(iris_df[['x', 'y']], clusterer=kmeans)
    
_ = interact(
    plot_kmeans,
    k=widgets.SelectionSlider(
        options=range(1, 10),
        description='Number of clusters (k)',
        layout=layout,
        style=style,
        orientation='horizontal'
    )
)

Auch hier erzeugen wir ein Silhouetten-Bild. Das Maximum liegt hier bei zwei Clustern. Dieser Wert scheint auf Basis der Daten auch visuell plausibel.

In [ ]:
sil_values = []
for k in range(2,10):
    kmeans = KMeans(n_clusters=k)
    clust = kmeans.fit_predict(iris_df[['x', 'y']])
    sil_values.append(silhouette_score(iris_df[['x', 'y']], clust))

fig, ax = plt.subplots()
ax.plot(range(2,10),sil_values)
ax.set_xlabel('k')
ax.set_xticks(range(2,10, 2))
ax.set_ylabel('Silhouette value');

### Die Auflösung

Tatsächlich entstammen die Daten jedoch *drei* verschiedenen Arten. Dies sehen wir, wenn wir die Spalte mit der Art hinzunehmen und die Punkte farbig kennzeichnen. Man sieht, dass die rote Art gut getrennt ist, jedoch die organene und graue Art ineinander übergehen. Diese Schwierigkeit bei der Separierung ist in der Praxis häufig anzutreffen.

In [ ]:
_ = iris_df.plot(kind='scatter',
            x='x',
            y='y',
            s=40,
            c='class',
            cmap=plt.get_cmap('Set1'),
            colorbar=False,
            figsize=(6, 6)
           )

## Vorteile und Nachteile von K-Means

| Vorteile | Nachteile |
| -------- | ----------|
| Sehr schnell und gut skalierbar | Empfindlich gegenüber Ausreißern |
| Neue Punkte können ohne Neutraining einem Cluster zugeordnet werden | Anzahl der Cluster muss vorgegeben werden |
| Clusterzuordnung kann als zusätzliches Feature für andere Modelle genutzt werden | Wenig flexibel in Bezug auf die Clustergröße: Alle Cluster sind mehr oder minder kugelförmig und ähnlich groß |
| Universell einsetzbar | Ergebnis hängt auch von der (zufälligen) Initialisierung ab &rarr; am besten mehrfach starten und die beste Variante wählen |
| Einfach zu verstehen | Clusterrepräsentanten (Zentroide) sind künstlich und nicht Teil der Daten |

# Hierarchisches Clustering

<img style="float: right; padding-left: 10%; padding-right: 5%;" src="images/hierarchical.png" width="55%">

### Agglomeratives Clustering
* Bottom-up
* Beginnt mit Clustern der Größe eins
* Cluster werden iterativ zusammengeführt  
&rarr; grüner Pfeil

### Aufspaltendes Clustering
* Top-down
* Beginnt mit allen Daten in einem Cluster
* Spaltet iterativ die Cluster auf  
&rarr; orangener Pfeil

<img style="float: left;" src="images/hierarchical_points.png" width="30%">

### Funktionsweise
* Wähle eine Distanzfunktion für Cluster (Linkage-Funktion)
* Jeder Datenpunkt startet als Einpunkt-Cluster
* Bestimme die beiden Cluster mit dem geringsten Wert der Linkage-Funktion und füge sie zusammen
* Wiederhole, bis nur noch ein Cluster übrig ist

<img style="float: right; padding-left: 10%; padding-right: 5%;" src="images/threshold.png" width="55%">

### Ergebnis
* Gibt eine Hierarchie von Clustern zurück
* Wird in einem Dendrogramm dargestellt
* Knoten sind Datenpunkte, jede Zeile entspricht einer Clusterzusammenfassung
* Gewünschte Clusteraufteilung erhält man, indem man eine Schwelle im Baum festlegt  
&rarr; z.B. mit dem "elbow heterogeneity criterion"

<img style="float: left;" src="images/threshold_clusters.png" width="30%">

### Linkage-Funktionen

Die Linkage-Funktion zur Zusammenführung der Cluster kann frei gewählt werden. Die beliebesten Linkage-Funktionen sind Single, Average, Complete und Ward Linkage.

<img src="images/linkage.png" width="80%">

Wir erproben nun das agglomerative Clustering mit dem Iris-Datensatz

In [ ]:
from sklearn.cluster import AgglomerativeClustering

def plot_hierarchical(k, linkage):
    agg = AgglomerativeClustering(n_clusters=k, linkage=linkage)
    fit_and_plot_clustering(iris_df[['x', 'y']], clusterer=agg)
    
_ = interact(
    plot_hierarchical,
    k=widgets.SelectionSlider(
        options=range(1, len(iris_df)),
        description='Number of clusters (k)',
        layout=layout,
        style=style,
        orientation='horizontal'
    ),
    linkage=widgets.Dropdown(
        options=['ward', 'complete', 'average', 'single'],
        layout=layout,
        style=style
    )
)

## Vorteile und Nachteile des agglomerativen Clusterings

| Vorteile | Nachteile |
| -------- | ----------|
| Kommt gut mit vielen Clustern zurecht | Berechnet die vollständige Hierarchie, auch wenn nur wenige Cluster gewünscht sind |
| Funktioniert auch bei unterschiedlich großen Clustern gut | Funktioniert nicht gut mit komplex geformten Clustern |
| Wahlmöglichkeit in Bezug auf Auswahl der Cluster und die Linkage-Funktion | Berechnung der Linkage-Funktion rechenaufwändig |
| Manche Implementierungen erlauben es, Vorwissen dazu einzubringen, welche Datenpunkte zu unterschiedlichen Clustern gehören | Bei Single-Linkage Neigung, Ketten zu bilden |
| Einfache Beurteilung des Clusterings in Abhängigkeit von der Heterogenitäts-Schwelle | Wahl einer bestimmten Heterogenitäts-Schwelle nötig |
| Ausreißer werden implizit identifiziert | Neue Punkte können nicht einem Cluster zugeordnet werden, ohne das Modell neu zu berechnen |

# Anhang

### Bessere Lösung für den Iris-Datensatz mit K-Means im Vierdimensionalen

In diesem Fall liegt es auch daran, dass wir nur zwei der beschreibenden Attribute im Clustering berücksichtigt haben - die 'sepal width (cm)' als 'x' sowie die 'petal length (cm)' als 'y', siehe die erste Code-Zelle unter Abschnitt 1.3. Diese beiden Dimensionen sind jene, die wir auch zum Zeichnen des zweidimensionalen Scatterplots herangezogen haben. Der k-Means Algorithmus ist jedoch sehr wohl in der Lage, mit deutlich mehr als zwei Dimensionen umzugehen. Der Datensatz enthält tatsächlich vier Dimensionen:

In [ ]:
iris_df.head()

Bauen wir also ein weiteres Cluster-Modell, welches alle vier Dimensionen berücksichtigt. Dabei setzen wir k=3, entsprechend der offiziellen Einteilung der Iris in drei Unterarten. 

In [ ]:
kmeans_all = KMeans(n_clusters=3) # Cluster
iris_df_cid = kmeans_all.fit_predict(iris_df[['s', 'x', 'y', 'p']]) # Nutze alle Attribute. Zuvor: Selektion von nur zwei Attributen mittels df[['x', 'y']]

Das resultierende Clustering trifft die Einteilung der Biologen sehr viel besser, wenn auch nicht perfekt (links die Einteilung der Biologen, rechts das Ergebnis des Clustermodells):

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
iris_df.plot(ax=axes[0],
        kind='scatter',
        x='x',
        y='y',
        s=40,
        c='class', # Farbe entsprechend der vorgegebenen 'class'
        cmap=plt.get_cmap('Set1'),
        colorbar=False,
        figsize=(14, 6)
        )
_ = iris_df.plot(ax=axes[1],
        kind='scatter',
        x='x',
        y='y',
        s=40,
        c=iris_df_cid, # Farbe entsprechend der berechneten Cluster-ID
        cmap=plt.get_cmap('Set1'),
        colorbar=False,
        figsize=(14, 6)
        )

Aus der bisher gewählten Projektion in die gewählten zwei Dimensionen x, y ist nicht ersichtlich, warum sich die grauen von den gelben Instanzen trennen lassen. Mit einer sogenannten Scatterplot-Matrix, welche systematisch alle möglichen Kombinationen von zwei (aus insgesamt vier) Dimensionen gegeneinander darstellt, lässt sich diese Trennung im hochdimensionalen Datenraum besser erkennen.

In [ ]:
scatter_matrix(iris_df[['s', 'x', 'y', 'p']],
               marker='o',
               alpha=1.0,
               c=iris_df_cid,
               cmap=plt.get_cmap('Set1'),
               figsize=(16, 16),
               diagonal="hist");

Aber auch mit vier Dimensionen liefert der Scatter-Plot nicht die von Biologen als richtig angesehene Trennung in drei Cluster

In [ ]:
sil_values = []
for k in range(2,10):
    kmeans = KMeans(n_clusters=k)
    clust = kmeans.fit_predict(iris_df[['s', 'x', 'y', 'p']])
    sil_values.append(silhouette_score(iris_df[['s', 'x', 'y', 'p']], clust))

fig, ax = plt.subplots()
ax.plot(range(2,10),sil_values)
ax.set_xlabel('k')
ax.set_xticks(range(2,10, 2))
ax.set_ylabel('Silhouette value');